In [1]:
import os
import astropy
from astropy.time import Time
from astropy import units
import pyuvdata
import datetime
import numpy as np

In [2]:
telescope_loc = pyuvdata.telescopes.known_telescope_location("OVRO-LWA")

In [3]:
observing_time = Time(datetime.datetime.now(), scale='utc', location=telescope_loc)
lst = observing_time.sidereal_time("apparent")
print(lst)

21h21m02.25217139s


In [4]:
lst_start = astropy.coordinates.Longitude(11, unit="hour")
print(lst_start)
lst_end = astropy.coordinates.Longitude(14, unit="hour")
print(lst_end)
print(lst > lst_start and lst < lst_end)

11h00m00s
14h00m00s
False


In [5]:
sun_loc = astropy.coordinates.get_sun(observing_time)
print(sun_loc)
sun_altaz = sun_loc.transform_to(astropy.coordinates.AltAz(obstime=observing_time, location=telescope_loc))
print(sun_altaz)
sun_altaz.alt

<SkyCoord (GCRS: obstime=2024-11-27 00:48:25.526056, obsgeoloc=(0., 0., 0.) m, obsgeovel=(0., 0., 0.) m / s): (ra, dec, distance) in (deg, deg, AU)
    (242.96799452, -21.11291565, 0.9867736)>
<SkyCoord (AltAz: obstime=2024-11-27 00:48:25.526056, location=(-2409247.20369628, -4477889.56236559, 3839327.82827939) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron): (az, alt, distance) in (deg, deg, AU)
    (245.43111027, -2.9077814, 0.9867757)>


<Latitude -2.9077814 deg>

In [6]:
10*units.deg

<Quantity 10. deg>

In [7]:
sun_altaz.alt > 10*units.deg

False

In [8]:
subband_dirs = np.sort([name for name in os.listdir("/lustre/pipeline/night-time") if name.endswith("MHz")])
print(subband_dirs)

['13MHz' '18MHz' '23MHz' '27MHz' '32MHz' '36MHz' '41MHz' '46MHz' '50MHz'
 '55MHz' '59MHz' '64MHz' '69MHz' '73MHz' '78MHz' '82MHz']


In [9]:
use_subband_dirs = [name for name in subband_dirs if int(name[:2]) >= 41]
print(use_subband_dirs)

['41MHz', '46MHz', '50MHz', '55MHz', '59MHz', '64MHz', '69MHz', '73MHz', '78MHz', '82MHz']


In [10]:
filenames = []
full_filepaths = []
lsts = []
in_lst_range = []
in_nighttime = []
for subband in use_subband_dirs:
    day_dirs = np.sort(os.listdir(f"/lustre/pipeline/night-time/{subband}"))
    for day in day_dirs:
        hours = np.sort(os.listdir(f"/lustre/pipeline/night-time/{subband}/{day}"))
        for hour in hours:
            files = np.sort(os.listdir(f"/lustre/pipeline/night-time/{subband}/{day}/{hour}"))
            for filename in files:
                file_time = datetime.datetime(
                    int(filename[:4]),  # year
                    int(filename[4:6]),  # month
                    int(filename[6:8]),  # day
                    int(filename[9:11]),  # hour
                    int(filename[11:13]),  # minute
                    int(filename[13:15]),  # second
                )
                observing_time = Time(file_time, scale='utc', location=pyuvdata.telescopes.known_telescope_location("OVRO-LWA"))
                lst = observing_time.sidereal_time("apparent")
                filenames.append(filename)
                full_filepaths.append(f"/lustre/pipeline/night-time/{subband}/{day}/{hour}/{filename}")
                lsts.append(lst)
                in_lst_range.append(lst > lst_start and lst < lst_end)
                sun_loc = astropy.coordinates.get_sun(observing_time)
                sun_altaz = sun_loc.transform_to(astropy.coordinates.AltAz(obstime=observing_time, location=telescope_loc))
                in_nighttime.append(sun_altaz.alt < 0)
                

KeyboardInterrupt: 

In [ ]:
print(in_nighttime)

In [ ]:
len(in_lst_range)

In [ ]:
if False:
    output_filename = f"/lustre/rbyrne/datafile_lsts.csv"
    with open(output_filename, "w") as f:
        f.write("filename, full path, LST, in LST range? (y/n) \n")
    with open(output_filename, "a") as f:
        f.write(
            f"{lst}, {model_file.removesuffix(f'_{use_band}MHz_source_sim.uvfits')} \n"
        )

In [ ]:
import casatasks

: 

In [12]:
casacore.concat()

AttributeError: module 'casacore' has no attribute 'concat'